# Light GBM

Light GBM, XGBoost'un eğitim süresi performansını artıramaya yönelik geliştirilen bir diğer GBM türüdür.

![image.png](image19.png)

LightGBM (Light Gradient Boosting Machine), Microsoft tarafından geliştirilen bir açık kaynaklı makine öğrenimi algoritmasıdır. GBM (Gradient Boosting Machine) algoritmasının daha hafif ve hızlı bir versiyonudur. Büyük veri setlerinde ve yüksek boyutlu özelliklerde mükemmel performans gösterir.

#### 🟢 **Level-wise Büyüme Stratejisi Yerine Leaf-wise Büyüme Stratejisi**
- Geleneksel GBM algoritmaları genellikle **Level-wise** büyüme stratejisini kullanır. Yani:
  - Ağaç derinlik seviyelerine göre genişler.
  - Her seviyedeki tüm yapraklar eşzamanlı olarak büyür.

- **LightGBM** ise **Leaf-wise** büyüme stratejisi kullanır:
  - En yüksek hataya sahip yapraktan başlayarak genişler.
  - Hataları daha hızlı azaltır.
  - Daha hızlı yakınsama sağlar.
  - Ancak **overfitting** riski yüksektir. Bu yüzden **`max_depth`** parametresiyle kontrol edilmelidir.

---

#### 🟢 **Breadth-First Search (BFS) Yerine Depth-First Search (DFS)**
- **BFS (Genişlik Öncelikli Arama):**
  - Seviyeleri sırayla genişletir (Level-wise).
  - Dengelidir ancak daha yavaştır.

- **DFS (Derinlik Öncelikli Arama):**
  - Yapraklardan başlar ve derinlik boyunca genişler (Leaf-wise).
  - Daha hızlıdır ve bellek açısından daha verimlidir.
  - LightGBM, **DFS tabanlı Leaf-wise büyüme** kullandığı için hız avantajı sağlar.

## Model

In [3]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=42)

In [4]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------- ----------- 1.0/1.5 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.9 MB/s eta 0:00:00


In [5]:
from lightgbm import LGBMRegressor

In [6]:
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 19
[LightGBM] [Info] Start training from score 543.483442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

## Tahmin

In [7]:
y_pred = lgbm_model.predict(X_test,
                   num_iteration=lgbm_model.best_iteration_)

In [8]:
np.sqrt(mean_squared_error(y_test, y_pred))

np.float64(363.8712087611089)

## Model Tuning

In [10]:
lgbm_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [11]:
lgbm_grid = {
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'n_estimators': [20, 40, 100, 200, 500, 1000],
    'max_depth': [1,2,3,4,5,6,7,8]
}

In [12]:
from sklearn.model_selection import GridSearchCV

lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=10, n_jobs=-1, verbose=2)

cv=10 parametresi 10-katlı çapraz doğrulama (10-fold cross validation) yapılacağını belirtir, yani veri seti 10 parçaya bölünüp, her seferinde 9 parça eğitim için, 1 parça test için kullanılarak model 10 kez eğitilip test edilecektir.

In [13]:
lgbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 19
[LightGBM] [Info] Start training from score 543.483442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

GridSearchCV(cv=10, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [20, 40, 100, 200, 500, 1000]},
             verbose=2)

In [14]:
lgbm_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20}

In [15]:
lgbm_tuned = LGBMRegressor(learning_rate=0.1,
                          max_depth=6,
                          n_estimators=20)
lgbm_tuned = lgbm_tuned.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 19
[LightGBM] [Info] Start training from score 543.483442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [20]:
y_pred = lgbm_tuned.predict(X_test)

In [21]:
np.sqrt(mean_squared_error(y_test, y_pred))

np.float64(371.5044868943621)